# **4 - Model**

## Imports

In [1]:
import time
import joblib
import itertools
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Predefined variables

In [3]:
CSV_SAVE_PATH = "../saved/"
NPZ_SAVE_PATH = "../saved/sign_language_preprocessed.npz"
MODEL_SAVE_PATH = "../model/"
SCALER_SAVE_PATH = "../model/"

## Load preprocessed data and verify

In [4]:
data = np.load(NPZ_SAVE_PATH, allow_pickle=True)
X_train, X_test, X_valid = data["X_train"], data["X_test"], data["X_valid"]  
y_train, y_test, y_valid = data["y_train"], data["y_test"], data["y_valid"]

print(f"Training set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Validation set: {X_valid.shape[0]} samples")
print(f"Number of classes: {len(np.unique(y_train))}")
print(f"Classes: {sorted(np.unique(y_train))}")

# Check label distribution
print(f"\nTraining label distribution:")
unique, counts = np.unique(y_train, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  {label}: {count} samples")

Training set: 1205 samples, 63 features
Test set: 61 samples
Validation set: 133 samples
Number of classes: 26
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

Training label distribution:
  A: 63 samples
  B: 35 samples
  C: 30 samples
  D: 39 samples
  E: 45 samples
  F: 49 samples
  G: 46 samples
  H: 33 samples
  I: 65 samples
  J: 65 samples
  K: 45 samples
  L: 61 samples
  M: 42 samples
  N: 50 samples
  O: 43 samples
  P: 36 samples
  Q: 42 samples
  R: 39 samples
  S: 56 samples
  T: 38 samples
  U: 40 samples
  V: 41 samples
  W: 45 samples
  X: 54 samples
  Y: 41 samples
  Z: 62 samples


## Train multiple models

### Define four different models to try and train

In [5]:
models = {
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "SVM": SVC(kernel='rbf', C=10, gamma='scale', random_state=42, probability=True),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

### Train the four models

In [6]:
trained_models = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    trained_models[name] = model
    print(f"{name} trained successfully!")

KNN trained successfully!
Random Forest trained successfully!
SVM trained successfully!
Neural Network trained successfully!


## Evaluate model performances

In [7]:
results_test = {}
results_valid = {}

print("Model Performance Results:")
print("-" * 70)
print(f"{'Model':<15} {'Test Accuracy':<15} {'Valid Accuracy':<15} {'Avg Accuracy':<15}")
print("-" * 70)

for name, model in trained_models.items():
    # Make predictions on test set
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    results_test[name] = test_accuracy
    
    # Make predictions on validation set  
    y_valid_pred = model.predict(X_valid)
    valid_accuracy = accuracy_score(y_valid, y_valid_pred)
    results_valid[name] = valid_accuracy
    
    # Calculate average
    avg_accuracy = (test_accuracy + valid_accuracy) / 2
    
    print(f"{name:<15} {test_accuracy:.4f} ({test_accuracy*100:.1f}%)   {valid_accuracy:.4f} ({valid_accuracy*100:.1f}%)    {avg_accuracy:.4f} ({avg_accuracy*100:.1f}%)")

# Find best model based on average of test and validation accuracy
avg_results = {name: (results_test[name] + results_valid[name]) / 2 for name in results_test}
best_model_name = max(avg_results, key=avg_results.get)
best_avg_accuracy = avg_results[best_model_name]

print("-" * 70)
print(f"Best Model: {best_model_name} with average accuracy: {best_avg_accuracy:.4f} ({best_avg_accuracy*100:.1f}%)")
print(f"Test accuracy: {results_test[best_model_name]:.4f} ({results_test[best_model_name]*100:.1f}%)")
print(f"Validation accuracy: {results_valid[best_model_name]:.4f} ({results_valid[best_model_name]*100:.1f}%)")

Model Performance Results:
----------------------------------------------------------------------
Model           Test Accuracy   Valid Accuracy  Avg Accuracy   
----------------------------------------------------------------------
KNN             0.6885 (68.9%)   0.6316 (63.2%)    0.6601 (66.0%)
Random Forest   0.7869 (78.7%)   0.7519 (75.2%)    0.7694 (76.9%)
SVM             0.8033 (80.3%)   0.7594 (75.9%)    0.7813 (78.1%)
Neural Network  0.8197 (82.0%)   0.8045 (80.5%)    0.8121 (81.2%)
----------------------------------------------------------------------
Best Model: Neural Network with average accuracy: 0.8121 (81.2%)
Test accuracy: 0.8197 (82.0%)
Validation accuracy: 0.8045 (80.5%)


### Note
Based on the above code cell, we can already see that the neural network has the best accuracy so far.

## Finetune the parameters of the four models

### Define ``grid_search`` function, given the model class, the parameter grid and the X & y train/test/valid

In [8]:
def grid_search(model_class, param_grid, X_train, y_train, X_test, y_test, X_valid, y_valid, model_name="Model"):
    print(f"\n{'='*50}")
    print(f"PARAMETER TUNING FOR {model_name.upper()}")
    print(f"{'='*50}")
    
    best_score = 0
    best_params = None
    best_model = None
    results = []
    
    # Generate all parameter combinations
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    
    total_combinations = 1
    for values in param_values:
        total_combinations *= len(values)
    
    print(f"Testing {total_combinations} parameter combinations...")
    print("-" * 80)
    print(f"{'Combination':<12} {'Parameters':<35} {'Test Acc':<10} {'Valid Acc':<10} {'Avg Acc':<10}")
    print("-" * 80)
    
    combination_num = 0

    # Test all combinations
    for param_combination in itertools.product(*param_values):
        combination_num += 1
        params = dict(zip(param_names, param_combination))
        
        try:
            # Create model with current parameters
            model = model_class(**params)
            
            # Train model
            start_time = time.time()
            model.fit(X_train, y_train)
            train_time = time.time() - start_time

            # Evaluate on test and validation sets
            test_pred = model.predict(X_test)
            valid_pred = model.predict(X_valid)
            
            test_acc = accuracy_score(y_test, test_pred)
            valid_acc = accuracy_score(y_valid, valid_pred)
            avg_acc = (test_acc + valid_acc) / 2

            # Store results
            results.append({
                'params': params,
                'test_acc': test_acc,
                'valid_acc': valid_acc,
                'avg_acc': avg_acc,
                'train_time': train_time,
                'model': model
            })
            
            # Update best model
            if avg_acc > best_score:
                best_score = avg_acc
                best_params = params
                best_model = model

            # Print results
            params_str = str(params)[:32] + "..." if len(str(params)) > 35 else str(params)
            print(f"{combination_num:<12} {params_str:<35} {test_acc:.4f}     {valid_acc:.4f}     {avg_acc:.4f}")
            
        except Exception as e:
            print(f"{combination_num:<12} ERROR: {str(e)[:50]}")
            continue
    
    print("-" * 80)
    print(f"\nBEST RESULTS FOR {model_name.upper()}:")
    print(f"Best parameters: {best_params}")
    print(f"Best average accuracy: {best_score:.4f} ({best_score*100:.2f}%)")
    print(f"Test accuracy: {[r['test_acc'] for r in results if r['params'] == best_params][0]:.4f}")
    print(f"Validation accuracy: {[r['valid_acc'] for r in results if r['params'] == best_params][0]:.4f}")
    
    return best_model, best_params, best_score, results

### Define parameter grids for the models

In [9]:
param_grids = {
    "KNN": {
        'n_neighbors': [1, 3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    },
    
    "Random Forest": {
        'n_estimators': [100, 200, 300, 400],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'random_state': [42]
    },
    
    "SVM": {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
        'kernel': ['rbf', 'poly', 'sigmoid'],
        'random_state': [42],
        'probability': [True]
    },
    
    "Neural Network": {
        'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100), (150, 100, 50)],
        'activation': ['relu', 'tanh', 'logistic'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive'],
        'max_iter': [500],
        'random_state': [42]
    }
}

# Print info over aantal combinaties
print("Number of parameter combinations to test per model:")
for model_name, grid in param_grids.items():
    total_combinations = 1
    for param_values in grid.values():
        total_combinations *= len(param_values)
    print(f"{model_name}: {total_combinations} combinations")

Number of parameter combinations to test per model:
KNN: 36 combinations
Random Forest: 144 combinations
SVM: 60 combinations
Neural Network: 90 combinations


### Start tuning

In [10]:
best_models_tuned = {}
all_tuning_results = {}

#### K-nearest Neighbor

In [11]:
print("Starting KNN parameter tuning...")

knn_best, knn_params, knn_score, knn_results = grid_search(
    KNeighborsClassifier, 
    param_grids["KNN"], 
    X_train, y_train, X_test, y_test, X_valid, y_valid,
    "KNN"
)

best_models_tuned["KNN"] = {
    'model': knn_best,
    'params': knn_params,
    'score': knn_score
}

all_tuning_results["KNN"] = knn_results

Starting KNN parameter tuning...

PARAMETER TUNING FOR KNN
Testing 36 parameter combinations...
--------------------------------------------------------------------------------
Combination  Parameters                          Test Acc   Valid Acc  Avg Acc   
--------------------------------------------------------------------------------
1            {'n_neighbors': 1, 'weights': 'u... 0.7705     0.6917     0.7311
2            ERROR: invalid literal for int() with base 10: 'A'
3            {'n_neighbors': 1, 'weights': 'u... 0.7705     0.6917     0.7311
4            {'n_neighbors': 1, 'weights': 'd... 0.7705     0.6917     0.7311
5            {'n_neighbors': 1, 'weights': 'd... 0.7377     0.6391     0.6884
6            {'n_neighbors': 1, 'weights': 'd... 0.7705     0.6917     0.7311
7            {'n_neighbors': 3, 'weights': 'u... 0.6885     0.6391     0.6638
8            ERROR: invalid literal for int() with base 10: 'A'
9            {'n_neighbors': 3, 'weights': 'u... 0.6885     0.63

#### Random Forest

In [12]:
print("Starting Random Forest parameter tuning...")

rf_best, rf_params, rf_score, rf_results = grid_search(
    RandomForestClassifier, 
    param_grids["Random Forest"], 
    X_train, y_train, X_test, y_test, X_valid, y_valid,
    "Random Forest"
)

best_models_tuned["Random Forest"] = {
    'model': rf_best,
    'params': rf_params,
    'score': rf_score
}

all_tuning_results["Random Forest"] = rf_results

Starting Random Forest parameter tuning...

PARAMETER TUNING FOR RANDOM FOREST
Testing 144 parameter combinations...
--------------------------------------------------------------------------------
Combination  Parameters                          Test Acc   Valid Acc  Avg Acc   
--------------------------------------------------------------------------------
1            {'n_estimators': 100, 'max_depth... 0.7541     0.7669     0.7605
2            {'n_estimators': 100, 'max_depth... 0.7049     0.7444     0.7246
3            {'n_estimators': 100, 'max_depth... 0.7213     0.7293     0.7253
4            {'n_estimators': 100, 'max_depth... 0.7705     0.7594     0.7649
5            {'n_estimators': 100, 'max_depth... 0.7869     0.7444     0.7656
6            {'n_estimators': 100, 'max_depth... 0.7213     0.7293     0.7253
7            {'n_estimators': 100, 'max_depth... 0.7213     0.7444     0.7328
8            {'n_estimators': 100, 'max_depth... 0.7377     0.7444     0.7410
9            {'

#### Support Vector Machine

In [13]:
print("Starting SVM parameter tuning...")

svm_best, svm_params, svm_score, svm_results = grid_search(
    SVC, 
    param_grids["SVM"], 
    X_train, y_train, X_test, y_test, X_valid, y_valid,
    "SVM"
)

best_models_tuned["SVM"] = {
    'model': svm_best,
    'params': svm_params,
    'score': svm_score
}

all_tuning_results["SVM"] = svm_results

Starting SVM parameter tuning...

PARAMETER TUNING FOR SVM
Testing 60 parameter combinations...
--------------------------------------------------------------------------------
Combination  Parameters                          Test Acc   Valid Acc  Avg Acc   
--------------------------------------------------------------------------------
1            {'C': 0.1, 'gamma': 'scale', 'ke... 0.3607     0.3158     0.3382
2            {'C': 0.1, 'gamma': 'scale', 'ke... 0.2131     0.2632     0.2381
3            {'C': 0.1, 'gamma': 'scale', 'ke... 0.1311     0.1579     0.1445
4            {'C': 0.1, 'gamma': 'auto', 'ker... 0.3443     0.3158     0.3300
5            {'C': 0.1, 'gamma': 'auto', 'ker... 0.2131     0.2256     0.2193
6            {'C': 0.1, 'gamma': 'auto', 'ker... 0.1311     0.1579     0.1445
7            {'C': 0.1, 'gamma': 0.001, 'kern... 0.0820     0.0602     0.0711
8            {'C': 0.1, 'gamma': 0.001, 'kern... 0.0656     0.0451     0.0553
9            {'C': 0.1, 'gamma': 0.0

#### Neural Network (MLP)

In [14]:
print("Starting Neural Network parameter tuning...")

nn_best, nn_params, nn_score, nn_results = grid_search(
    MLPClassifier, 
    param_grids["Neural Network"], 
    X_train, y_train, X_test, y_test, X_valid, y_valid,
    "Neural Network"
)

best_models_tuned["Neural Network"] = {
    'model': nn_best,
    'params': nn_params,
    'score': nn_score
}

all_tuning_results["Neural Network"] = nn_results

Starting Neural Network parameter tuning...

PARAMETER TUNING FOR NEURAL NETWORK
Testing 90 parameter combinations...
--------------------------------------------------------------------------------
Combination  Parameters                          Test Acc   Valid Acc  Avg Acc   
--------------------------------------------------------------------------------
1            {'hidden_layer_sizes': (50,), 'a... 0.7705     0.8045     0.7875
2            {'hidden_layer_sizes': (50,), 'a... 0.7705     0.8045     0.7875
3            {'hidden_layer_sizes': (50,), 'a... 0.7869     0.7895     0.7882
4            {'hidden_layer_sizes': (50,), 'a... 0.7869     0.7895     0.7882
5            {'hidden_layer_sizes': (50,), 'a... 0.8033     0.7970     0.8001
6            {'hidden_layer_sizes': (50,), 'a... 0.8033     0.7970     0.8001
7            {'hidden_layer_sizes': (50,), 'a... 0.7869     0.8045     0.7957
8            {'hidden_layer_sizes': (50,), 'a... 0.7869     0.8045     0.7957
9            {

### Compare the tuned models with the original models

In [15]:
print("="*80)
print("COMPARISON: ORIGINAL vs TUNED MODELS")
print("="*80)
print(f"{'Model':<15} {'Original Acc':<15} {'Tuned Acc':<15} {'Improvement':<15} {'Best Parameters'}")
print("-"*80)

original_scores = {}
for name, model in trained_models.items():
    test_acc = accuracy_score(y_test, model.predict(X_test))
    valid_acc = accuracy_score(y_valid, model.predict(X_valid))
    original_scores[name] = (test_acc + valid_acc) / 2

for model_name in best_models_tuned.keys():
    original_acc = original_scores.get(model_name, 0)
    tuned_acc = best_models_tuned[model_name]['score']
    improvement = tuned_acc - original_acc
    best_params = best_models_tuned[model_name]['params']
    
    params_str = str(best_params)[:40] + "..." if len(str(best_params)) > 43 else str(best_params)
    
    print(f"{model_name:<15} {original_acc:.4f}          {tuned_acc:.4f}          {improvement:+.4f}          {params_str}")

print("-"*80)


best_tuned_name = max(best_models_tuned.keys(), key=lambda x: best_models_tuned[x]['score'])
best_tuned_score = best_models_tuned[best_tuned_name]['score']

print(f"\nBEST TUNED MODEL: {best_tuned_name}")
print(f"Score: {best_tuned_score:.4f} ({best_tuned_score*100:.2f}%)")
print(f"Parameters: {best_models_tuned[best_tuned_name]['params']}")


original_best_name = max(original_scores.keys(), key=lambda x: original_scores[x])
original_best_score = original_scores[original_best_name]

print(f"\nIMPROVEMENT OVER ORIGINAL BEST:")
print(f"Original best: {original_best_name} ({original_best_score:.4f})")
print(f"Tuned best: {best_tuned_name} ({best_tuned_score:.4f})")
print(f"Overall improvement: {best_tuned_score - original_best_score:+.4f} ({((best_tuned_score - original_best_score)/original_best_score)*100:+.2f}%)")

COMPARISON: ORIGINAL vs TUNED MODELS
Model           Original Acc    Tuned Acc       Improvement     Best Parameters
--------------------------------------------------------------------------------
KNN             0.6601          0.7311          +0.0711          {'n_neighbors': 1, 'weights': 'uniform',...
Random Forest   0.7694          0.7694          +0.0000          {'n_estimators': 200, 'max_depth': None,...
SVM             0.7813          0.8015          +0.0202          {'C': 100, 'gamma': 'scale', 'kernel': '...
Neural Network  0.8121          0.8510          +0.0389          {'hidden_layer_sizes': (100, 50), 'activ...
--------------------------------------------------------------------------------

BEST TUNED MODEL: Neural Network
Score: 0.8510 (85.10%)
Parameters: {'hidden_layer_sizes': (100, 50), 'activation': 'tanh', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 500, 'random_state': 42}

IMPROVEMENT OVER ORIGINAL BEST:
Original best: Neural Network (0.8121)
Tuned

### Detailed analysis of the best model

In [20]:
best_model = best_models_tuned[best_tuned_name]['model']

print(f"DETAILED EVALUATION OF BEST MODEL: {best_tuned_name}")
print("="*60)

y_test_pred = best_model.predict(X_test)
y_valid_pred = best_model.predict(X_valid)

test_accuracy = accuracy_score(y_test, y_test_pred)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)

print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"Validation Accuracy: {valid_accuracy:.4f} ({valid_accuracy*100:.2f}%)")
print(f"Average Accuracy: {(test_accuracy + valid_accuracy)/2:.4f}")

print(f"\nBest Parameters: {best_models_tuned[best_tuned_name]['params']}")

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred))

DETAILED EVALUATION OF BEST MODEL: Neural Network
Test Accuracy: 0.8525 (85.25%)
Validation Accuracy: 0.8496 (84.96%)
Average Accuracy: 0.8510

Best Parameters: {'hidden_layer_sizes': (100, 50), 'activation': 'tanh', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 500, 'random_state': 42}

Classification Report (Test Set):
              precision    recall  f1-score   support

           A       1.00      1.00      1.00         1
           B       0.75      1.00      0.86         3
           C       1.00      0.67      0.80         3
           D       1.00      1.00      1.00         1
           F       1.00      1.00      1.00         2
           G       1.00      1.00      1.00         3
           H       1.00      1.00      1.00         2
           I       1.00      1.00      1.00         2
           J       1.00      1.00      1.00         4
           K       1.00      0.67      0.80         3
           M       1.00      1.00      1.00         2
           N    

## Select and save best model

### Save the best model

In [25]:
best_model = best_models_tuned[best_tuned_name]['model']

model_filename = MODEL_SAVE_PATH + f"{best_tuned_name.replace(' ', '_').lower()}_model.pkl"
joblib.dump(best_model, model_filename)

print(f"Best model ({best_tuned_name}) saved as: {model_filename}")
print(f"Model accuracy: {best_tuned_score:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Validation accuracy: {valid_accuracy:.4f}")

Best model (Neural Network) saved as: ../model/neural_network_model.pkl
Model accuracy: 0.8510
Test accuracy: 0.8525
Validation accuracy: 0.8496


### Load the scaler

In [26]:
try:
    scaler = data["scaler"].item()
    print("Scaler loaded from preprocessed data file")

except Exception as e:
    print(f"Scaler not found in preprocessed data ({e}), recreating...")
    
    # Recreate scaler from training data (same approach as preprocessing)
    try:
        df_original = pd.read_csv(f'{CSV_SAVE_PATH}sign_language_train.csv')  # Use train data only for fitting scaler
        X_original = df_original.iloc[:, :-1].values
        y_original = df_original.iloc[:, -1].values
        
        # Apply same preprocessing as in preprocessing notebook
        X_reshaped = X_original.reshape(-1, 21, 3)
        
        def center_hand(landmarks):
            wrist = landmarks[0]
            centered = landmarks - wrist
            return centered
        
        def scale_hand(landmarks):
            wrist = landmarks[0]
            middle_finger_tip = landmarks[12]
            scale = np.linalg.norm(middle_finger_tip - wrist)
            if scale > 0:
                scaled = landmarks / scale
            else:
                scaled = landmarks
            return scaled
        
        X_centered = np.array([center_hand(hand) for hand in X_reshaped])
        X_scaled = np.array([scale_hand(hand) for hand in X_centered])
        X_processed = X_scaled.reshape(-1, 63)
        
        # Create and fit scaler
        scaler = StandardScaler()
        scaler.fit(X_processed)
        print("Scaler recreated and fitted on training data")

    except Exception as e2:
        print(f"Error recreating scaler: {e2}")
        scaler = None

Scaler loaded from preprocessed data file


### Save scaler separately

In [27]:
if scaler is not None:
    scaler_filename = SCALER_SAVE_PATH + "scaler.pkl"
    joblib.dump(scaler, scaler_filename)
    print(f"Scaler saved as: {scaler_filename}")
else:
    print("Could not save scaler")

Scaler saved as: ../model/scaler.pkl
